# Создание различных ML моделей для определения спам сообщений

## Импорт необходимых библиотек

> Ячейки с установкой библиотек были приведены к типу `Raw`, чтобы они не выполнялись при автоматическом запуске всех ячеек.

In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import pickle

In [2]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import nltk
from nltk.corpus import stopwords

In [3]:
import sys
sys.path.append('../utils')  # Добавление папки ../utils в список путей поиска модулей.
from preprocessing import count_tags, count_links, count_emojis, count_whitespaces, count_newlines, replace_tags, replace_links, remove_emojis, preprocess_text

## Чтение обработанного датасета

In [4]:
df = pd.read_csv('../data/preprocessed.csv', index_col=0)

In [5]:
df.head()

,text,label,emojis,newlines,whitespaces,links,tags,text_preprocessed
0,Добрый день! Отличается ли перечень необходимы...,0,0,0,0,0,0,добрый день отличается ли перечень необходимых...
1,Узбекистан. Рассматриваются обе формы,0,0,0,0,0,0,узбекистан рассматриваются обе формы
2,"Здравствуйте, а как проходит поступление после...",0,0,0,0,0,0,здравствуйте а как проходит поступление после ...
3,Спасибо большое за ответ!,0,0,0,0,0,0,спасибо большое за ответ
4,"Здравствуйте, а когда будет день открытых двер...",0,0,0,0,0,0,здравствуйте а когда будет день открытых двере...


## Подготовка выборок

In [6]:
texts = list(df['text_preprocessed'])  # Тексты
labels = list(df['label'])  # Метки (целевые значения)

In [7]:
additional_features = list(df[['emojis', 'whitespaces', 'links', 'tags']].values)  # Дополнительные числовые признаки

### Разделение на обучающую и тестовую выборки

In [8]:
X_train_text, X_test_text, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

In [9]:
X_train_num, X_test_num = train_test_split(
    additional_features, test_size=0.2, random_state=42, stratify=labels
)

### Подготовка стоп-слов

Загрузка английских и русских стоп-слов (`nltk`)

In [10]:
nltk.download('stopwords')
russian_stopwords = set(stopwords.words('russian'))
english_stopwords = set(ENGLISH_STOP_WORDS)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\overklassniy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Объединение русских и английских стоп-слов

In [11]:
combined_stopwords = list(russian_stopwords.union(english_stopwords))

In [12]:
combined_stopwords

['both',
 'already',
 'get',
 'many',
 'no',
 'thereafter',
 'enough',
 'ли',
 'his',
 'serious',
 'until',
 'опять',
 'чтобы',
 'such',
 'у',
 'from',
 'done',
 'onto',
 'кто',
 'across',
 'other',
 'though',
 'none',
 'тебя',
 'hereby',
 'on',
 'move',
 'whole',
 'was',
 'а',
 'со',
 'то',
 'at',
 'together',
 'нибудь',
 'можно',
 'via',
 'as',
 'hereafter',
 'так',
 'нас',
 'behind',
 'made',
 'ним',
 'than',
 'beside',
 'eleven',
 'cant',
 'чуть',
 'several',
 'при',
 'whence',
 'most',
 'toward',
 'разве',
 'put',
 'мы',
 'много',
 'between',
 'ей',
 'describe',
 'об',
 'how',
 'alone',
 'ему',
 'раз',
 'everywhere',
 'up',
 'after',
 'seeming',
 'sometime',
 'someone',
 'detail',
 'there',
 'whereas',
 'interest',
 'нет',
 'hundred',
 'latterly',
 'eg',
 'такой',
 'перед',
 'за',
 'ie',
 'seems',
 'during',
 'themselves',
 'this',
 'что',
 'наконец',
 'however',
 'him',
 'без',
 'because',
 'himself',
 'again',
 'wherein',
 'am',
 'herself',
 'therein',
 'been',
 'больше',
 'why'

Проверка на наличие слов `tag` и `link` в списке стоп-слов.

In [13]:
'TAG' in combined_stopwords, 'tag' in combined_stopwords, 'LINK' in combined_stopwords, 'link' in combined_stopwords

(False, False, False, False)

In [14]:
len(combined_stopwords)

469

### Подготовка текстовых и числовых признаков

#### Преобразование текста в числовой формат (TF-IDF)

In [15]:
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    stop_words=combined_stopwords,
    min_df=2,
    sublinear_tf=True
)

In [16]:
text_features_train = vectorizer.fit_transform(X_train_text)
text_features_test = vectorizer.transform(X_test_text)

#### Масштабирование числовых признаков

In [17]:
numerical_features_train = pd.DataFrame(X_train_num, columns=['emojis', 'whitespaces', 'links', 'tags'])
numerical_features_test = pd.DataFrame(X_test_num, columns=['emojis', 'whitespaces', 'links', 'tags'])

scaler = StandardScaler()
numerical_features_train_scaled = scaler.fit_transform(numerical_features_train)
numerical_features_test_scaled = scaler.transform(numerical_features_test)

#### Объединение текстовых и числовых признаков

In [18]:
X_train = hstack([text_features_train, numerical_features_train_scaled])
X_test = hstack([text_features_test, numerical_features_test_scaled])

#### Дополнительная синтетическая генерация

In [19]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

## Обучение моделей

### Logistic Regression

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE 

In [20]:
# 4. Обучение модели Logistic Regression
model_lr = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

### Logistic Regression + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [21]:
# 4. Обучение модели Logistic Regression
model_lr_sm = LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)
model_lr_sm.fit(X_train_resampled, y_train_resampled)

LogisticRegression(class_weight='balanced', max_iter=500, random_state=42)

### Random Forest

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

In [22]:
model_rf = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

### Random Forest + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка с SMOTE

In [23]:
model_rf_sm = RandomForestClassifier(class_weight='balanced', random_state=42, n_estimators=100)
model_rf_sm.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(class_weight='balanced', random_state=42)

### Naive Bayes

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

Объединение текстов и числовых признаков без масштабирования

In [24]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

Обучение Naive Bayes

In [25]:
model_nb = MultinomialNB()
model_nb.fit(X_train_nc, y_train)

MultinomialNB()

### Naive Bayes + SMOTE

- Стоп слова NLTK английские + русские  
- Выборка без SMOTE

Объединение текстов и числовых признаков без масштабирования

In [26]:
X_train_nc = hstack([text_features_train, X_train_num])
X_test_nc = hstack([text_features_test, X_test_num])

In [27]:
smote = SMOTE(random_state=42)
X_train_resampled_nc, y_train_resampled_nc = smote.fit_resample(X_train_nc, y_train)

In [28]:
model_nb_sm = MultinomialNB()
model_nb_sm.fit(X_train_resampled_nc, y_train_resampled_nc)

MultinomialNB()

## Оценка качества моделей

Примеры тестовых сообщений

In [29]:
test_messages = [
    "Это честное сообщение от пользователя.",
    "🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎",
    "Зарабатывай миллионы **онлайн** прямо сейчас!",
    "Работа на дому, легкий доход. Пиши в личку!",
    "Привет! Как дела? У меня всё отлично.",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM\n\nhttps://t.me/ancleroyofficial",
    "Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or click the link on my bio",
    "steam gift 50$ - steamcommunity.com/gift-card/pay/50\n@everyone",
    "Давайте **вместе** будем писать про казино в чатах!!! Присоединяйтесь!",
    "Как же надоели эти сообщения про казино",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: stankin.ru",
    "Добрый день. Для подачи документов необходимо пройти регистрацию здесь: https://stankin.ru",
    "Поступление – это почти что казино! Лотерея!"
]

Функция для предсказания на новых данных

> Эта же функция находится в `../utils/preprocessing.py`.

In [30]:
def predict_message(text: str, vectorizer, scaler, model) -> tuple[int, list[float]]:
    """
    Предсказывает метку и вероятности для текста с использованием модели машинного обучения.

    Параметры:
        text: (str) - входной текст для анализа
        vectorizer: - векторизатор для преобразования текста
        scaler: - масштабатор числовых признаков
        model: - модель машинного обучения

    Возвращает:
        tuple[int, list[float]] - метка предсказания и вероятности
    """
    # Предварительная обработка текста
    text_preprocessed = preprocess_text(text)

    # Векторизация текста
    text_vector = vectorizer.transform([text_preprocessed])

    # Создание числовых признаков
    numerical_features = pd.DataFrame(
        [[count_emojis(text), count_whitespaces(text), count_links(text), count_tags(text)]],
        columns=['emojis', 'whitespaces', 'links', 'tags']
    )
    # Масштабирование числовых признаков
    numerical_features_scaled = scaler.transform(numerical_features)

    # Объединение текстовых и числовых признаков
    features = hstack([text_vector, numerical_features_scaled])

    # Предсказание
    prediction = model.predict(features)
    probabilities = model.predict_proba(features)
    return int(prediction[0]), *probabilities.tolist()

### Logistic Regression

In [31]:
y_pred = model_lr.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00      4948
           1       0.64      0.89      0.75        55

    accuracy                           0.99      5003
   macro avg       0.82      0.94      0.87      5003
weighted avg       0.99      0.99      0.99      5003

Confusion Matrix:
 [[4921   27]
 [   6   49]]


In [32]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9899234, 0.0100766]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0479845, 0.9520155]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.875087, 0.124913]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.6023825, 0.3976175]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9710339, 0.0289661]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.001

### Logistic Regression + SMOTE

In [33]:
y_pred = model_lr_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      1.00      4948
           1       0.66      0.87      0.75        55

    accuracy                           0.99      5003
   macro avg       0.83      0.93      0.87      5003
weighted avg       0.99      0.99      0.99      5003

Confusion Matrix:
 [[4923   25]
 [   7   48]]


In [34]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_lr_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9906417, 0.0093583]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.017223, 0.982777]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.7672135, 0.2327865]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.2622872, 0.7377128]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.97795, 0.02205]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.0006886

### Random Forest

In [35]:
y_pred = model_rf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4948
           1       0.74      0.62      0.67        55

    accuracy                           0.99      5003
   macro avg       0.87      0.81      0.83      5003
weighted avg       0.99      0.99      0.99      5003

Confusion Matrix:
 [[4936   12]
 [  21   34]]


In [36]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [1.0, 0.0]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.69, 0.31]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9981812, 0.0018188]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.858312, 0.141688]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.99, 0.01]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.36, 0.64]

Сообщение: Discover the 

### Random Forest + SMOTE

In [37]:
y_pred = model_rf_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4948
           1       0.97      0.65      0.78        55

    accuracy                           1.00      5003
   macro avg       0.98      0.83      0.89      5003
weighted avg       1.00      1.00      1.00      5003

Confusion Matrix:
 [[4947    1]
 [  19   36]]





In [38]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_rf_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [1.0, 0.0]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.59, 0.41]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9995364, 0.0004636]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.93, 0.07]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [1.0, 0.0]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.22, 0.78]

Сообщение: Discover the hidden sec

### Naive Bayes

In [39]:
y_pred = model_nb.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      4948
           1       0.61      0.67      0.64        55

    accuracy                           0.99      5003
   macro avg       0.80      0.83      0.82      5003
weighted avg       0.99      0.99      0.99      5003

Confusion Matrix:
 [[4924   24]
 [  18   37]]





In [40]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9989309, 0.0010691]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 0
Вероятности: [0.7165656, 0.2834344]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 0
Вероятности: [0.9895535, 0.0104465]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 0
Вероятности: [0.9185113, 0.0814887]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9995681, 0.0004319]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [4.7

### Naive Bayes + SMOTE

In [41]:
y_pred = model_nb_sm.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print('\n\n')

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99      4948
           1       0.32      0.96      0.48        55

    accuracy                           0.98      5003
   macro avg       0.66      0.97      0.74      5003
weighted avg       0.99      0.98      0.98      5003

Confusion Matrix:
 [[4837  111]
 [   2   53]]





In [42]:
for message in test_messages:
    prediction, probabilities = predict_message(message, vectorizer, scaler, model_nb_sm)
    print(f"Сообщение: {message}")
    print(f"Класс: {prediction}")  # 0 = не реклама, 1 = реклама
    print(f"Вероятности: {[float(f'{prob:.7f}') for prob in probabilities]}\n")

Сообщение: Это честное сообщение от пользователя.
Класс: 0
Вероятности: [0.9959097, 0.0040903]

Сообщение: 🔥 Казино онлайн! Зарабатывай миллионы прямо сейчас! 💰💎
Класс: 1
Вероятности: [0.0001502, 0.9998498]

Сообщение: Зарабатывай миллионы **онлайн** прямо сейчас!
Класс: 1
Вероятности: [0.3689199, 0.6310801]

Сообщение: Работа на дому, легкий доход. Пиши в личку!
Класс: 1
Вероятности: [0.1286178, 0.8713822]

Сообщение: Привет! Как дела? У меня всё отлично.
Класс: 0
Вероятности: [0.9988315, 0.0011685]

Сообщение: Discover the hidden secrets of the digital market that top traders don’t want you to know! I’m seeking five motivated individuals who are committed to earning over $100K weekly in the digital market. Once you start seeing profits, I’ll require just 15% of your earnings as my fee. Please note: I’m only interested in working with five serious and dedicated people should send me a direct message or ask me (HOW) via TELEGRAM

https://t.me/ancleroyofficial
Класс: 1
Вероятности: [0.0

## Сохранение результатов

In [43]:
with open('../models/vectorizer.pkl', 'wb') as vec_file:
    pickle.dump(vectorizer, vec_file)

with open('../models/scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

print("Векторизатор и масштабатор сохранены!")

Векторизатор и масштабатор сохранены!


In [44]:
with open('../models/lr_model.pkl', 'wb') as lr_model_file:
    pickle.dump(model_lr, lr_model_file)

print("Модель логической регрессии без семплирования сохранена!")

Модель логической регрессии без семплирования сохранена!


In [45]:
with open('../models/lr_sm_model.pkl', 'wb') as lr_sm_model_file:
    pickle.dump(model_lr_sm, lr_sm_model_file)

print("Модель логической регрессии со семплированием сохранена!")

Модель логической регрессии со семплированием сохранена!


In [46]:
with open('../models/rf_model.pkl', 'wb') as rf_model_file:
    pickle.dump(model_rf, rf_model_file)

print("Модель случайного леса без семплирования сохранена!")

Модель случайного леса без семплирования сохранена!


In [47]:
with open('../models/rf_sm_model.pkl', 'wb') as rf_sm_model_file:
    pickle.dump(model_rf_sm, rf_sm_model_file)

print("Модель случайного леса со семплированием сохранена!")

Модель случайного леса со семплированием сохранена!


In [48]:
with open('../models/nb_model.pkl', 'wb') as nb_model_file:
    pickle.dump(model_nb, nb_model_file)

print("Модель Naive Bayes без семплирования сохранена!")

Модель Naive Bayes без семплирования сохранена!


In [49]:
with open('../models/nb_sm_model.pkl', 'wb') as nb_sm_model_file:
    pickle.dump(model_nb_sm, nb_sm_model_file)

print("Модель Naive Bayes со семплированием сохранена!")

Модель Naive Bayes со семплированием сохранена!
